In [10]:
from chat import *
import dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory

from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain.chains.combine_documents import create_stuff_documents_chain

from typing import Dict
from langchain_core.runnables import RunnablePassthrough

In [11]:
dotenv.load_dotenv()

True

In [12]:
doc_chain, retriever = create_docchain_retriever("./data/review_sample.csv")
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Answer the user's questions based on the below context:\n\n{context}")), MessagesPlaceholder(variable_name='messages')])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x164335b40>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x164336950>, model_name='gpt-3.5-turbo-1106', openai_api_key=SecretStr('**********'), o

In [13]:
retrieval_chain = create_retrieval_chain(doc_chain, retriever)
retrieval_chain

RunnableAssign(mapper={
  context: RunnableLambda(parse_retriever_input)
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x16551a3e0>)
})
| RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
    context: RunnableLambda(format_docs)
  }), config={'run_name': 'format_inputs'})
  | ChatPromptTemplate(input_variables=['context', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Answer the user's questions based on the below context:\n\n{context}")), MessagesPlaceholder(variable_name='messages')])
  | ChatOpenAI(

In [14]:
# User asks first question in chat
chat_history = ChatMessageHistory()

chat_history.add_user_message("how can Camilla Grill improve themselves?")

response = retrieval_chain.invoke(
    {
        "messages": chat_history.messages,
    },
)
response

'Camilla Grill can improve by focusing on offering healthier menu options alongside their classic greasy-spoon favorites. They could also enhance their customer service and overall dining experience to attract a wider range of customers. Additionally, incorporating sustainable and locally-sourced ingredients could help them appeal to the growing market of environmentally-conscious consumers.'

In [15]:
chat_history.add_ai_message(response)

In [16]:
chat_history.add_user_message("based on that response, what are two ways to enhance their customer service?")

In [17]:
response = retrieval_chain.invoke(
    {
        "messages": chat_history.messages,
    },
)
response

'To enhance their customer service, Camilla Grill could implement staff training programs focused on attentive and responsive service. They could also encourage a customer feedback system to actively listen to and address any concerns or suggestions from patrons.'